#### Data 612
#### Project 1
#### Kevin DiPerna

This system recommends movies to users

Create my dataset and user item matrix

In [101]:
import pandas as pd
import numpy as np

data = {
    'user': ['Alice', 'Alice', 'Alice', 'Alice', 'Alice',
             'Bob', 'Bob', 'Bob', 'Bob', 'Bob',
             'Charlie', 'Charlie', 'Charlie', 'Charlie', 'Charlie',
             'David', 'David', 'David', 'David', 'David',
             'Eve', 'Eve', 'Eve', 'Eve', 'Eve'],
    'movie': ['Inception', 'Matrix', 'Avengers', 'Joker', 'Titanic'] * 5,
    'rating': [
        5, 4, np.nan, 4, np.nan,
        3, 5, 4, np.nan, 1,
        4, 3, 2, 4, 2,
        4, np.nan, 5, np.nan, 4,
        np.nan, 3, 4, 2, 5
    ]
}

df = pd.DataFrame(data)

# Create user-item matrix
user_item_matrix = df.pivot_table(index='user', columns='movie', values='rating')

print(user_item_matrix)

movie    Avengers  Inception  Joker  Matrix  Titanic
user                                                
Alice         NaN        5.0    4.0     4.0      NaN
Bob           4.0        3.0    NaN     5.0      1.0
Charlie       2.0        4.0    4.0     3.0      2.0
David         5.0        4.0    NaN     NaN      4.0
Eve           4.0        NaN    2.0     3.0      5.0


Break into training and test set and find the raw average with training set

In [102]:
from sklearn.model_selection import train_test_split

df_nonan = df.dropna()

train_df, test_df = train_test_split(df_nonan, test_size=0.3, random_state=42)

raw_average = train_df['rating'].mean()
print(raw_average)

3.5384615384615383


Calculate the RSME for train and test set

In [103]:
train_df['raw average'] = raw_average
test_df['raw average'] = raw_average

from sklearn.metrics import mean_squared_error
import numpy as np

train_rmse = np.sqrt(mean_squared_error(train_df['rating'], train_df['raw average']))

test_rmse = np.sqrt(mean_squared_error(test_df['rating'], test_df['raw average']))

print(f"Train RMSE (baseline): {train_rmse}")
print(f"Test RMSE (baseline): {test_rmse}")

Train RMSE (baseline): 1.2162606385262997
Test RMSE (baseline): 0.9514859136040755


Find the average rating for each user and movie. Find the bias and baseline predictor

In [104]:

user_avg = train_df.groupby('user')['rating'].mean().rename('user_avg')
user_avg_df = user_avg.reset_index(name='user_avg')
user_avg_df['bias'] = user_avg_df['user_avg'] - 3.5384615384615383

# Average rating for each movie
movie_avg = train_df.groupby('movie')['rating'].mean().rename('movie_avg')
movie_avg_df = movie_avg.reset_index(name='movie_avg')
movie_avg_df['bias'] = movie_avg_df['movie_avg'] - 3.5384615384615383


In [97]:
baseline_df = user_avg_df[['user', 'bias']].merge(
    movie_avg_df[['movie', 'bias']], how='cross'
)

raw_average = 3.5384615384615383
baseline_df['baseline_pred'] = raw_average + baseline_df['bias_x'] + baseline_df['bias_y']

# Step 3: Rename columns for clarity (optional)
baseline_df = baseline_df.rename(columns={
    'bias_x': 'user_bias',
    'bias_y': 'movie_bias'
})

baseline_df.head(10)

,user,user_bias,movie,movie_bias,baseline_pred
0,Alice,0.461538,Avengers,-0.038462,3.961538
1,Alice,0.461538,Inception,0.128205,4.128205
2,Alice,0.461538,Joker,-0.205128,3.794872
3,Alice,0.461538,Matrix,0.461538,4.461538
4,Alice,0.461538,Titanic,-0.205128,3.794872
5,Bob,-0.538462,Avengers,-0.038462,2.961538
6,Bob,-0.538462,Inception,0.128205,3.128205
7,Bob,-0.538462,Joker,-0.205128,2.794872
8,Bob,-0.538462,Matrix,0.461538,3.461538
9,Bob,-0.538462,Titanic,-0.205128,2.794872


Reformat the dataframes

In [98]:
train_df = train_df.merge(
    baseline_df[['user', 'movie', 'baseline_pred']],
    on=['user', 'movie'],
    how='left'
)
print(train_df)

       user      movie  rating  raw average  baseline_pred
0       Bob  Inception     3.0     3.538462       3.128205
1     David   Avengers     5.0     3.538462       4.294872
2       Eve     Matrix     3.0     3.538462       3.794872
3       Eve      Joker     2.0     3.538462       3.128205
4     Alice      Joker     4.0     3.538462       3.794872
5   Charlie   Avengers     2.0     3.538462       3.294872
6       Eve    Titanic     5.0     3.538462       3.128205
7       Bob     Matrix     5.0     3.538462       3.461538
8     David  Inception     4.0     3.538462       4.461538
9   Charlie  Inception     4.0     3.538462       3.461538
10  Charlie      Joker     4.0     3.538462       3.128205
11    David    Titanic     4.0     3.538462       4.128205
12      Bob    Titanic     1.0     3.538462       2.794872


In [99]:
test_df = test_df.merge(
    baseline_df[['user', 'movie', 'baseline_pred']],
    on=['user', 'movie'],
    how='left'
)
print(test_df)

      user      movie  rating  raw average  baseline_pred
0    Alice  Inception     5.0     3.538462       4.128205
1      Bob   Avengers     4.0     3.538462       2.961538
2  Charlie    Titanic     2.0     3.538462       3.128205
3    Alice     Matrix     4.0     3.538462       4.461538
4  Charlie     Matrix     3.0     3.538462       3.794872
5      Eve   Avengers     4.0     3.538462       3.294872


Find the training and test RSME

In [100]:
train_rmse = np.sqrt(mean_squared_error(train_df['rating'], train_df['baseline_pred']))

test_rmse = np.sqrt(mean_squared_error(test_df['rating'], test_df['baseline_pred']))

print(f"Train RMSE (baseline predictor): {train_rmse:.4f}")
print(f"Test RMSE (baseline predictor): {test_rmse:.4f}")

Train RMSE (baseline predictor): 1.0564
Test RMSE (baseline predictor): 0.8615


Summary - The baseline predictor RMSE for the train set is 1.0564 and for the test set is 0.8615 suggesting that the test set is better at predicting more accurately. The baseline predictors RMSE is lower than the RMSE using raw average which is expected since we are factoring in user and movie bias.